All the Jupyter Notebooks are available at https://github.com/neuro-ml/dpipe_tutorial

# Tutorials on Deep Pipe

The tutorials introduce the library called **Deep Pipe**, which is useful for medical image analysis, including preprocessing, data augmentation, performance validation and final prediction.

## Tutorial 3: Batch Iterators

In the current tutorial we take a look at batch iterators from `dpipe`.

### Imports:

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


# import deep pipe library
# how to install: https://github.com/neuro-ml/deep_pipe/blob/master/README.md
import dpipe

/nmnt/media/home/shmulev/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## 0. Parallel Data Processing (PDP) 
*Source: https://pypi.python.org/pypi/pdp/0.2.1*

Example from the repository:

In [2]:
import time
import numpy as np

from pdp import One2One, Many2One, Source, Pipeline, pack_args, \
    combine_batches

Generate patient indices:

In [3]:
patient_ids = [f'patient_{i}' for i in range(5)] * 10

Define fake function to "load" our images:

In [4]:
def load_data(patient_id):
    x = np.random.randn(2, 100, 100, 100)
    y = np.random.randn(100, 100, 100)
    return x, y

# Pack args for all functions that have multiple arguments
@pack_args
def augment(x, y):
    x = x * (1 + 0.1 * np.random.randn(*x.shape))
    return x, y

In [5]:
batch_size = 20

pipeline = Pipeline(
    Source(patient_ids, buffer_size=10),
    One2One(load_data, buffer_size=10),
    One2One(augment, buffer_size=10, n_workers=1),
    Many2One(batch_size, buffer_size=10),
    One2One(combine_batches, buffer_size=3)
)

with pipeline:
    for x, y in pipeline:
        time.sleep(0.1)
    x_ = x
del x, y

In [6]:
x_.shape

(20, 2, 100, 100, 100)

## I. `Slices`

In [7]:
from dpipe.batch_iter.slices import slices
from dpipe.train.batch_iter import make_batch_iter_from_finite
from dpipe.dataset.brats import Brats2017
from dpipe.medim.visualize import slice3d
from functools import partial

In [8]:
dataset = Brats2017(data_path='/nmnt/t01-ssd/brats2017/train')

In [9]:
batch_iter = make_batch_iter_from_finite(
    get_batch_iter=partial(
        slices,
        ids=dataset.ids[:1],
        load_x=dataset.load_image,
        load_y=dataset.load_msegm,
        slices=1,
        batch_size=16,
        shuffle=False
    )
)

In [10]:
xs, ys = [], []
with batch_iter:
    for x, y in batch_iter:
            xs.append(x)
            ys.append(y)

In [11]:
xs[0].shape

(16, 4, 240, 240)

In [12]:
dataset.load_image(dataset.ids[0]).shape

(4, 240, 240, 155)

In [13]:
slice3d(xs[0][0], axis=0)

A Jupyter Widget